### Chainer でクラス分類

### データの読み込み

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("wine-class.csv")

In [5]:
df.head()

,Class,Alcohol,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Color intensity,Hue,Proline
0,1,14.23,2.43,15.6,127,2.80,3.06,0.28,5.64,1.04,1065
1,1,13.20,2.14,11.2,100,2.65,2.76,0.26,4.38,1.05,1050
2,1,13.16,2.67,18.6,101,2.80,3.24,0.30,5.68,1.03,1185
3,1,14.37,2.50,16.8,113,3.85,3.49,0.24,7.80,0.86,1480
4,1,13.24,2.87,21.0,118,2.80,2.69,0.39,4.32,1.04,735


 ### 入力変数と教師データ（出力変数）に切り分ける

In [13]:
# df.loc[行, 列]
t = df.iloc[:, 0].values -1 # 1〜3 のラベルを 0〜2 にできる
x = df.iloc[:, 1:].values

In [15]:
t.dtype

dtype('int64')

In [16]:
x.dtype

dtype('float64')

In [17]:
# chainer で使うために 32 ビットに変換する
t = t.astype("i")
x = x.astype("f")

In [18]:
t.dtype

dtype('int32')

In [19]:
x.dtype

dtype('float32')

### ニューラルネットワークのモデルを定義

In [35]:
import chainer
import chainer.functions as F
import chainer.links as L
import numpy as np

In [32]:
class NN(chainer.Chain):

    # モデルの構造を明示
    def __init__(self, n_mid_units = 5, n_out = 3):
        super().__init__()
        with self.init_scope():
            self.fc1 = L.Linear(None, n_mid_units) # 第一引数を None にすると自動で入力の数を把握してくれる。
            self.fc2 = L.Linear(None, n_out)

    # 順伝播
    def __call__(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h # h => hidden_layer

In [36]:
np.random.seed(1)
# インスタンス化
nn = NN()
model = L.Classifier(nn)

### Chainer で使用するデータセットの形式

In [37]:
dataset = list(zip(x, t))

### 訓練データと検証データに分割

In [38]:
# 訓練データを７割、検証データを３割
n_train = int(len(dataset) * 0.7)
train, test = chainer.datasets.split_dataset_random(dataset, n_train, seed=1)

### 学習するための設定

#### optimizer の設定

In [39]:
optimizer = chainer.optimizers.SGD() # 確率的勾配降下法

In [41]:
optimizer.setup(model)

#### Iterator の設定

In [43]:
batchsize = 10
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

#### updater の設定
- CPU：device=-1
- GPU：device=0

In [44]:
from chainer import training

In [46]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)

#### Trainer と Extensions の設定

In [47]:
from chainer.training import extensions

In [48]:
# エポックの数
epoch = 50 # 50 回学習を繰り返す

In [52]:
# trainer の宣言
trainer = training.Trainer(updater, (epoch, 'epoch'), out = 'result/wine')

# 検証データで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))
# 1エポックごとに trainデータに対する loss と testデータに対する loss
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))